In [0]:
# Para a quinta visualização é preciso duas tabelas.
# Será feito um gráfico de network.
# Então será necessário uma tabela para as conexões e outra para os nós.

# O sexto gráfico é gerado com os mesmo dados das conexões. 

In [0]:
import pandas as pd
import numpy as np

In [0]:
alunos_caxias = pd.read_csv('alunos_caxias_2018.csv')
alunos_caxias.head()

,ID_ALUNO,NU_ANO_CENSO,CO_IES,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,NO_CURSO,NU_IDADE,ANO_INGRESSO,TP_SITUACAO,TP_NIVEL_ACADEMICO,TP_SEXO
0,71CCAB7D5BAE6603B3995CA2EB8F9534,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,34,2014,Cursando,Graduação,Masculino
1,FBC7BF40B3F9AC1C46F78CFF70AC141B,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,28,2018,Cursando,Graduação,Masculino
2,43ABA22A4C7DC2F6E69C71CE90D339A0,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,27,2017,Matrícula trancada,Graduação,Masculino
3,10707CB293B32AD1BC23EB641765BCE7,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,28,2014,Matrícula trancada,Graduação,Feminino
4,7C95F1E0E536524DC3281C4452030F72,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,54,2007,Matrícula trancada,Graduação,Feminino


In [0]:
# selecionar os registros com a situação 'Cursando' e que começaram na IES em 2018
situacao = alunos_caxias.TP_SITUACAO == 'Cursando' # situação cursando
nivel_academico = (alunos_caxias.TP_NIVEL_ACADEMICO == 'Graduação') #nível de graduação presencial e a distância 
ano = (alunos_caxias.ANO_INGRESSO == 2018)
cursando = alunos_caxias.loc[situacao & nivel_academico & ano,:].copy() # alunos cursando em 2018
cursando.CO_IES.value_counts() # alunos que se matricularam e estão cursando em 2018 em cada IES

Centro Universitário Da Serra Gaúcha                       4370
Universidade De Caxias Do Sul                              3001
Centro Universitário Uniftec                               1859
Faculdade Anhanguera De Caxias Do Sul                       410
Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul     111
Faculdade Murialdo                                          100
Faculdade Nossa Senhora De Fátima                           100
Faculdade Ideau De Caxias Do Sul                             78
Name: CO_IES, dtype: int64

In [0]:
# códigos que não estão vinculados a algum curso em 2018 (desvinculado do curso ou matrícula trancada)
anos = (alunos_caxias.ANO_INGRESSO.isin([2016,2017,2018])) # só interessa registros recentes
situacao = (alunos_caxias.TP_SITUACAO.isin(['Desvinculado do curso', 'Matrícula trancada', 'Transferido para outro curso da mesma IES']))
nivel_academico = (alunos_caxias.TP_NIVEL_ACADEMICO == 'Graduação')

nao_cursando = alunos_caxias.loc[anos & situacao & nivel_academico,:].copy()
nao_cursando.CO_IES.value_counts() 
# estudantes que iniciaram nas respectivas IES entre 2016 e 2018, mas sairam do curso

Universidade De Caxias Do Sul                              5181
Centro Universitário Da Serra Gaúcha                       3359
Centro Universitário Uniftec                               1096
Faculdade Anhanguera De Caxias Do Sul                       602
Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul     210
Faculdade Murialdo                                          194
Faculdade Ideau De Caxias Do Sul                            120
Faculdade Nossa Senhora De Fátima                            90
Name: CO_IES, dtype: int64

In [0]:
cursando.head()

,ID_ALUNO,NU_ANO_CENSO,CO_IES,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,NO_CURSO,NU_IDADE,ANO_INGRESSO,TP_SITUACAO,TP_NIVEL_ACADEMICO,TP_SEXO
1,FBC7BF40B3F9AC1C46F78CFF70AC141B,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,28,2018,Cursando,Graduação,Masculino
31,CBDDDDED425BFD6FE4D0FE4FF02C51A3,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,30,2018,Cursando,Graduação,Feminino
66,38DE9F2553152BC720F5883A1CCA2F1F,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,27,2018,Cursando,Graduação,Masculino
112,9329D565D05B90609DA9CD08965CEC09,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,25,2018,Cursando,Graduação,Feminino
116,437C230EC285FF3C87C3403B0D609B35,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,24,2018,Cursando,Graduação,Feminino


In [0]:
nao_cursando.head()

,ID_ALUNO,NU_ANO_CENSO,CO_IES,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,NO_CURSO,NU_IDADE,ANO_INGRESSO,TP_SITUACAO,TP_NIVEL_ACADEMICO,TP_SEXO
2,43ABA22A4C7DC2F6E69C71CE90D339A0,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,27,2017,Matrícula trancada,Graduação,Masculino
24,06358262B67CC5FC6B9DD6A79B3279C6,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,49,2016,Matrícula trancada,Graduação,Feminino
30,828155F75E4535A19FCCD13968A8351F,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,34,2016,Matrícula trancada,Graduação,Masculino
42,5739B97B827D9861EA3DC2CCA708F892,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,35,2017,Matrícula trancada,Graduação,Masculino
51,E5978EE98829531087113382015BC3EF,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,27,2018,Desvinculado do curso,Graduação,Feminino


In [0]:
def renomear_colunas(data, mapa):
  '''
  renomear as colunas para montar o antes e o 
  depois dos alunos
  
  argumentos:
    data -> pandas dataframe
    mapa -> dicionário
  '''
  colunas = ['ID_ALUNO',
             'CO_IES',
             'NO_CURSO',
             'TP_SITUACAO',
             'ANO_INGRESSO',
             'TP_GRAU_ACADEMICO',
             'TP_MODALIDADE_ENSINO']
  
  data = data[colunas].copy()
  data.rename(columns = mapa, inplace = True)
  return data

In [0]:
# É preciso renomear os nomes das colunas para poder juntar os dois dataframe da situação anterior e atual do aluno. 

colunas = ['CO_IES',
           'NO_CURSO',
           'TP_SITUACAO',
           'ANO_INGRESSO',
           'TP_GRAU_ACADEMICO',
           'TP_MODALIDADE_ENSINO']

novas_nao_cursando = ['ies_anterior',
                      'curso_anterior',
                      'situacao_anterior',
                      'ano_anterior',
                      'grau_anterior',
                      'modalidade_anterior']
novas_cursando = ['ies_atual',
                  'curso_atual',
                  'situacao_atual',
                  'ano_atual',
                  'grau_atual',
                  'modalidade_atual']

mapa_nao_cursando = dict(zip(colunas, novas_nao_cursando))
mapa_cursando = dict(zip(colunas, novas_cursando))

mapa_cursando, mapa_nao_cursando

({'ANO_INGRESSO': 'ano_atual',
  'CO_IES': 'ies_atual',
  'NO_CURSO': 'curso_atual',
  'TP_GRAU_ACADEMICO': 'grau_atual',
  'TP_MODALIDADE_ENSINO': 'modalidade_atual',
  'TP_SITUACAO': 'situacao_atual'},
 {'ANO_INGRESSO': 'ano_anterior',
  'CO_IES': 'ies_anterior',
  'NO_CURSO': 'curso_anterior',
  'TP_GRAU_ACADEMICO': 'grau_anterior',
  'TP_MODALIDADE_ENSINO': 'modalidade_anterior',
  'TP_SITUACAO': 'situacao_anterior'})

In [0]:
# dataframe da situação anterior
antes = renomear_colunas(nao_cursando, mapa_nao_cursando)
antes.head()

,ID_ALUNO,ies_anterior,curso_anterior,situacao_anterior,ano_anterior,grau_anterior,modalidade_anterior
2,43ABA22A4C7DC2F6E69C71CE90D339A0,Universidade De Caxias Do Sul,História,Matrícula trancada,2017,Licenciatura,Presencial
24,06358262B67CC5FC6B9DD6A79B3279C6,Universidade De Caxias Do Sul,História,Matrícula trancada,2016,Licenciatura,Presencial
30,828155F75E4535A19FCCD13968A8351F,Universidade De Caxias Do Sul,História,Matrícula trancada,2016,Licenciatura,Presencial
42,5739B97B827D9861EA3DC2CCA708F892,Universidade De Caxias Do Sul,História,Matrícula trancada,2017,Licenciatura,Presencial
51,E5978EE98829531087113382015BC3EF,Universidade De Caxias Do Sul,História,Desvinculado do curso,2018,Licenciatura,Presencial


In [0]:
# dataframe da situação depois
depois = renomear_colunas(cursando, mapa_cursando)
depois.head()

,ID_ALUNO,ies_atual,curso_atual,situacao_atual,ano_atual,grau_atual,modalidade_atual
1,FBC7BF40B3F9AC1C46F78CFF70AC141B,Universidade De Caxias Do Sul,História,Cursando,2018,Licenciatura,Presencial
31,CBDDDDED425BFD6FE4D0FE4FF02C51A3,Universidade De Caxias Do Sul,História,Cursando,2018,Licenciatura,Presencial
66,38DE9F2553152BC720F5883A1CCA2F1F,Universidade De Caxias Do Sul,História,Cursando,2018,Licenciatura,Presencial
112,9329D565D05B90609DA9CD08965CEC09,Universidade De Caxias Do Sul,História,Cursando,2018,Licenciatura,Presencial
116,437C230EC285FF3C87C3403B0D609B35,Universidade De Caxias Do Sul,História,Cursando,2018,Licenciatura,Presencial


In [0]:
# transição dos estudantes entre cursos da mesma IES ou IES distintas
students_flow = antes.merge(depois, on = 'ID_ALUNO')
students_flow.head(10)

,ID_ALUNO,ies_anterior,curso_anterior,situacao_anterior,ano_anterior,grau_anterior,modalidade_anterior,ies_atual,curso_atual,situacao_atual,ano_atual,grau_atual,modalidade_atual
0,6EE8F10AE6C23604BA9E66B50CBE8BFA,Universidade De Caxias Do Sul,História,Matrícula trancada,2016,Licenciatura,Presencial,Centro Universitário Da Serra Gaúcha,1427646,Cursando,2018,Tecnólogo,a distância
1,E18340E6B025CF3D7926D03F8B8A3655,Universidade De Caxias Do Sul,História,Matrícula trancada,2016,Licenciatura,Presencial,Centro Universitário Uniftec,Marketing,Cursando,2018,Tecnólogo,Presencial
2,2179F94C8A6F7ECE96775F3918D43A28,Universidade De Caxias Do Sul,História,Matrícula trancada,2016,Licenciatura,Presencial,Centro Universitário Da Serra Gaúcha,Relações Internacionais,Cursando,2018,Bacharelado,Presencial
3,2179F94C8A6F7ECE96775F3918D43A28,Centro Universitário Da Serra Gaúcha,Arquitetura E Urbanismo,Transferido para outro curso da mesma IES,2017,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Relações Internacionais,Cursando,2018,Bacharelado,Presencial
4,2E0E5FBF1352C5418152C9A0A55934F3,Universidade De Caxias Do Sul,Serviço Social,Desvinculado do curso,2017,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Direito,Cursando,2018,Bacharelado,Presencial
5,9AFC2902070B9B4E917AABBA7D3D161E,Universidade De Caxias Do Sul,Serviço Social,Transferido para outro curso da mesma IES,2017,Bacharelado,Presencial,Universidade De Caxias Do Sul,Direito,Cursando,2018,Bacharelado,Presencial
6,83BF48541638A5D1B3B38029E17B7266,Universidade De Caxias Do Sul,Direito,Desvinculado do curso,2016,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Direito,Cursando,2018,Bacharelado,Presencial
7,C411DFC070DDF83CB05E9F900E54891D,Universidade De Caxias Do Sul,Direito,Desvinculado do curso,2018,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Relações Internacionais,Cursando,2018,Bacharelado,Presencial
8,0F6434A66497D8E501D9214244410CE9,Universidade De Caxias Do Sul,Direito,Desvinculado do curso,2016,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Direito,Cursando,2018,Bacharelado,Presencial
9,DF4307F06DAF517C538658B9A0018A67,Universidade De Caxias Do Sul,Direito,Matrícula trancada,2017,Bacharelado,Presencial,Centro Universitário Da Serra Gaúcha,Relações Internacionais,Cursando,2018,Bacharelado,Presencial


In [0]:
# salvar em csv
students_flow.to_csv('fluxo_estudantes_2018.csv', index = False)

In [0]:
# gerar os agrupamentos entre a IES anterior e a atual dos alunos.
fluxo = students_flow.groupby(["ies_anterior", "ies_atual"]).size()
fluxo

ies_anterior                                             ies_atual                                              
Centro Universitário Da Serra Gaúcha                     Centro Universitário Da Serra Gaúcha                       364
                                                         Centro Universitário Uniftec                                75
                                                         Faculdade Anhanguera De Caxias Do Sul                       28
                                                         Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul     26
                                                         Faculdade Ideau De Caxias Do Sul                             4
                                                         Faculdade Murialdo                                          13
                                                         Faculdade Nossa Senhora De Fátima                           10
                                               

In [0]:
siglas = np.load('dicionario_ies_siglas.npy', allow_pickle=True).item()
siglas

{'Centro Universitário Da Serra Gaúcha': 'FSG',
 'Centro Universitário Uniftec': 'FTEC',
 'Faculdade América Latina': 'América Latina',
 'Faculdade Anglo-Americano De Caxias Do Sul': 'Anglo-Americano',
 'Faculdade Angloamericano De Caxias Do Sul': 'Anglo-Americano',
 'Faculdade Anhanguera De Caxias Do Sul': 'Anhanguera',
 'Faculdade Da Serra Gaucha': 'FSG',
 'Faculdade Da Serra Gaúcha': 'FSG',
 'Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul': 'FTSG',
 'Faculdade De Tecnologia Tecbrasil': 'FTEC',
 'Faculdade Dos Imigrantes  Fai': 'FAI',
 'Faculdade Dos Imigrantes - Fai': 'FAI',
 'Faculdade Ideau De Caxias Do Sul': 'IDEAU',
 'Faculdade La Salle - Caxias': 'La Salle',
 'Faculdade Montserrat': 'América Latina',
 'Faculdade Murialdo': 'Murialdo',
 'Faculdade Nossa Senhora De Fatima': 'Fátima',
 'Faculdade Nossa Senhora De Fátima': 'Fátima',
 'Faculdade Serrana': 'IDEAU',
 'Universidade De Caxias Do Sul': 'UCS'}

In [0]:
# Montagem dos dados para colocar no flourish
fluxo_ies = pd.DataFrame(list(fluxo.keys()), columns = ['De Onde', 'Para Onde'])
fluxo_ies['Quantidade'] = fluxo.values

# remover grupos iguais. Exemplo: ucs e ucs; fsg e fsg (esses são alunos que trocaram de curso, mas não de IES)

fluxo_ies = fluxo_ies.groupby(by = ['De Onde','Para Onde']).filter(lambda x: x['De Onde'] != x['Para Onde'])

# trocar os nomes das IES por siglas usando o dicionario 'siglas'

fluxo_ies['De Onde'].replace(siglas, inplace = True)
fluxo_ies['Para Onde'].replace(siglas, inplace = True)

fluxo_ies

,De Onde,Para Onde,Quantidade
1,FSG,FTEC,75
2,FSG,Anhanguera,28
3,FSG,FTSG,26
4,FSG,IDEAU,4
5,FSG,Murialdo,13
6,FSG,Fátima,10
7,FSG,UCS,118
8,FTEC,FSG,55
10,FTEC,Anhanguera,7
11,FTEC,FTSG,2


In [0]:
# csv
fluxo_ies.to_csv('quinta_e_sexta_visualizacoes_conexoes.csv', index = False)

In [0]:
# criação da tabela dos nós

cursando_2018 = cursando.CO_IES.value_counts()
cursando_2018

Centro Universitário Da Serra Gaúcha                       4370
Universidade De Caxias Do Sul                              3001
Centro Universitário Uniftec                               1859
Faculdade Anhanguera De Caxias Do Sul                       410
Faculdade De Tecnologia Da Serra Gaúcha - Caxias Do Sul     111
Faculdade Murialdo                                          100
Faculdade Nossa Senhora De Fátima                           100
Faculdade Ideau De Caxias Do Sul                             78
Name: CO_IES, dtype: int64

In [0]:
iniciaram_2018 = pd.DataFrame(zip(cursando_2018.keys(), cursando_2018.values),
                             columns = ['Nome da IES','Iniciaram em 2018'])
iniciaram_2018['Nome da IES'].replace(siglas, inplace = True)

# Logos das instituições de ensino superior de Caxias do Sul
logos = ['https://upload.wikimedia.org/wikipedia/commons/4/4d/LogotipoFSG.png',
         'https://www.ucs.br/site/static/img/capa2016/logo_ucs.png',
         'https://www.ftec.com.br/static/front_end/_images/geral/logo-grupo.png',
         'https://i1.wp.com/tempofm.com.br/wp-content/uploads/2018/10/anhanguera-2.jpg?resize=945%2C630',
         'https://qb-assets.querobolsa.com.br/logos/colorido/large/159/logo_1489433583.png',
         'https://www.sinprocaxias.com.br/igc/uploadAr/FileProcessingScripts/PHP/UploadedFiles/thumbs/faculdade_fatima-sinprocaxias.png',
         'https://www.faculdadefatima.com.br/vestibular/images/logo.png',
         'https://www.caxias.ideau.com.br/']

iniciaram_2018['logos'] =  logos

iniciaram_2018

,Nome da IES,Iniciaram em 2018,logos
0,FSG,4370,https://upload.wikimedia.org/wikipedia/commons...
1,UCS,3001,https://www.ucs.br/site/static/img/capa2016/lo...
2,FTEC,1859,https://www.ftec.com.br/static/front_end/_imag...
3,Anhanguera,410,https://i1.wp.com/tempofm.com.br/wp-content/up...
4,FTSG,111,https://qb-assets.querobolsa.com.br/logos/colo...
5,Murialdo,100,https://www.sinprocaxias.com.br/igc/uploadAr/F...
6,Fátima,100,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,IDEAU,78,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [0]:
def gerar_texto(dados, fluxo, coluna):
  '''
  função para gerar a coluna com os textos
  representando o fluxo dos alunos
  
  parâmetros:
    dados -> pandas dataframe a ser usado
    fluxo -> pandas dataframe com os dados dos fluxos dos alunos
    coluna -> nome da coluna em que estão os nomes da IES
  '''
  ies = list(dados[coluna])
  string = []
  for cada in ies:
    if cada == 'UCS':
      string.append('')
    else:
      valor1 = fluxo.loc[(fluxo['De Onde']==cada) & (fluxo['Para Onde']=='UCS'),'Quantidade'].values
      valor2 = fluxo.loc[(fluxo['De Onde']=='UCS') & (fluxo['Para Onde']==cada),'Quantidade'].values
      
      if valor1.size == 0: valor1 = 0  
      else: valor1 = valor1[0]
      if valor2.size == 0: valor2 = 0  
      else: valor2 = valor2[0]
      
      texto = '{} -> UCS: {}'.format(cada, valor1) + '<br/>' + 'UCS -> {}: {}'.format(cada, valor2)
      string.append(texto)
      
  return pd.Series(string)

In [0]:
# adicionando a coluna de texto
iniciaram_2018['texto'] = gerar_texto(iniciaram_2018, fluxo_ies, 'Nome da IES')
iniciaram_2018

,Nome da IES,Iniciaram em 2018,logos,texto
0,FSG,4370,https://upload.wikimedia.org/wikipedia/commons...,FSG -> UCS: 118<br/>UCS -> FSG: 302
1,UCS,3001,https://www.ucs.br/site/static/img/capa2016/lo...,
2,FTEC,1859,https://www.ftec.com.br/static/front_end/_imag...,FTEC -> UCS: 15<br/>UCS -> FTEC: 41
3,Anhanguera,410,https://i1.wp.com/tempofm.com.br/wp-content/up...,Anhanguera -> UCS: 4<br/>UCS -> Anhanguera: 10
4,FTSG,111,https://qb-assets.querobolsa.com.br/logos/colo...,FTSG -> UCS: 0<br/>UCS -> FTSG: 3
5,Murialdo,100,https://www.sinprocaxias.com.br/igc/uploadAr/F...,Murialdo -> UCS: 8<br/>UCS -> Murialdo: 3
6,Fátima,100,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",Fátima -> UCS: 1<br/>UCS -> Fátima: 2
7,IDEAU,78,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",IDEAU -> UCS: 0<br/>UCS -> IDEAU: 3


In [0]:
# csv
iniciaram_2018.to_csv('quinta_visualizacao_nos.csv', index = False)